In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [3]:
stats = pd.read_excel('COMBINED_NORMAL_ADVANCED_STATS_1984-85_to_2019-20_ver2.xlsx', 0)
salaries = pd.read_excel('salaries_1985to2020_final.xlsx')
salary_caps = pd.read_excel('salary_caps.xlsx')

In [4]:
file2 = pd.ExcelFile("2019_20_FINAL_normal_advanced_stats_V2.xlsx")
data_2020 = pd.read_excel(file2, 0)

In [5]:
stats = stats.drop(columns=['slug', 'key'])
data_2020 = data_2020.drop(columns=['Unnamed: 0'])

In [6]:
all([stats.columns.values[i]==data_2020.columns.values[i] for i in range(len(stats.columns.values))])

True

In [8]:
stats = stats.rename(columns = {'Year': 'season'})
salaries = salaries.rename(columns = {"names":'name'})

In [9]:
train = stats.merge(salaries, on = ['name', 'season'])

In [11]:
salary_caps = salary_caps.drop(columns=['Unnamed: 0'])
salary_caps.head()

,season,salary_cap
0,2027-28,167513000
1,2026-27,159536000
2,2025-26,151939000
3,2024-25,144704000
4,2023-24,137813000


In [16]:
train_with_sc = train.merge(salary_caps, on = ['season'])

In [18]:
individual_salaries = train_with_sc['salary']
salary_caps1 = train_with_sc['salary_cap_x']

In [19]:
cap_usage = individual_salaries/salary_caps1

In [20]:
train_with_sc['cap_usage'] = cap_usage

In [24]:
train_with_sc[['true_shooting_percentage', 'win_shares', 'cap_usage']]

,true_shooting_percentage,win_shares,cap_usage
0,0.628,11.2,0.425000
1,0.578,6.8,0.138889
2,0.563,7.2,0.222222
3,0.570,6.6,0.111111
4,0.524,4.6,0.058056
...,...,...,...
14870,0.611,3.9,0.132799
14871,0.000,0.0,0.018985
14872,0.640,0.3,0.018985
14873,0.633,1.8,0.015166


In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

### BUILD MODEL HERE

In [96]:
def model_builder(df, features, response = 'cap_usage'):
    """df: dataframe with stats, salaries, and cap usage
    features: is a list of metrics for our regression model to predict salary usage."""
    df_features = df[features]
    X = df_features
    #X = StandardScaler().fit_transform(X)
    y = df[[response]]
    reg = LinearRegression().fit(X, y)
    print('The coefficients are {}'.format(reg.coef_))
    return reg

def predict_player(test_df, model, name_of_player, features):
    """player_features: must be the same metrics used to build the model in a df"""
    #parameters = model.get_params
    print(model.get_params)
    player_only = test_df[test_df['name'] == name_of_player]
    ready_for_test = player_only[features]
    prediction = model.predict(ready_for_test)[0][0]
    return [name_of_player, prediction]

def predict_all(test_df, model, features):
    test_df_clean = test_df.dropna()
    players = test_df_clean['name'].values
    ready_for_test = test_df_clean[features]
    predictions = model.predict(ready_for_test)
    print('The mean of cap usage predictions is {}'.format(np.mean(predictions)))
    #print(players.shape, predictions.shape)
    return [[players[i], predictions[i, 0]] for i in range(len(players))]

In [97]:
def do_all(train_df, test_df, features, response = 'cap_usage', predict_all_bool = False, player_names = ['Steven Adams']):
    model = model_builder(train_df, features, response)
    if predict_all_bool:
        return predict_all(test_df, model, features)
    predictions = []
    for player in player_names:
        predictions.append(predict_player(test_df, model, player, features))
    return predictions

In [77]:
train_with_sc.columns

Index(['name', 'positions', 'age', 'team_x', 'games_played', 'games_started',
       'minutes_played', 'made_field_goals', 'attempted_field_goals',
       'made_three_point_field_goals', 'attempted_three_point_field_goals',
       'made_free_throws', 'attempted_free_throws', 'offensive_rebounds',
       'defensive_rebounds', 'assists', 'steals', 'blocks', 'turnovers',
       'personal_fouls', 'points', 'player_efficiency_rating',
       'true_shooting_percentage', 'three_point_attempt_rate',
       'free_throw_attempt_rate', 'offensive_rebound_percentage',
       'defensive_rebound_percentage', 'total_rebound_percentage',
       'assist_percentage', 'steal_percentage', 'block_percentage',
       'turnover_percentage', 'usage_percentage', 'offensive_win_shares',
       'defensive_win_shares', 'win_shares', 'win_shares_per_48_minutes',
       'offensive_box_plus_minus', 'defensive_box_plus_minus',
       'box_plus_minus', 'value_over_replacement_player', 'season',
       'player_id', 'sa

In [104]:
rishab_and_alvin_best = ['value_over_replacement_player', 'points', 'player_efficiency_rating', 'box_plus_minus', 'steal_percentage', 'age']
from_team = ['value_over_replacement_player', 'win_shares', 'player_efficiency_rating', 'minutes_played', 'box_plus_minus']

### RUN MODEL HERE

In [105]:
do_all(train_with_sc, data_2020, from_team, predict_all_bool = True)

The coefficients are [[ 1.73833562e-02 -1.92990639e-03  6.89539203e-04  3.35632649e-03
  -7.83282235e-05]]
The mean of cap usage predictions is 0.07001378518028742


[['Steven Adams', 0.12033359571379525],
 ['Bam Adebayo', 0.1603153912651193],
 ['LaMarcus Aldridge', 0.13480974527126144],
 ['Kyle Alexander', 0.018659527388554496],
 ['Nickeil Alexander-Walker', 0.035070181307642216],
 ['Grayson Allen', 0.06408393577402328],
 ['Jarrett Allen', 0.11482805870371689],
 ['Kadeem Allen', 0.041635027907748104],
 ['Al-Farouq Aminu', 0.0672132653237549],
 ['Justin Anderson', 0.02828928561711383],
 ['Kyle Anderson', 0.07904184775061103],
 ['Ryan Anderson', 0.02304589296928252],
 ['Giannis Antetokounmpo', 0.20928666161366388],
 ['Kostas Antetokounmpo', 0.017377254085803113],
 ['Thanasis Antetokounmpo', 0.02294378950322286],
 ['Carmelo Anthony', 0.09535483335428721],
 ['OG Anunoby', 0.1207639449019439],
 ['Ryan Arcidiacono', 0.04679847552326105],
 ['Trevor Ariza', 0.098990034557815],
 ['Trevor Ariza', 0.08538049245294753],
 ['Trevor Ariza', 0.11632249584036354],
 ['D.J. Augustin', 0.08408196636115467],
 ['Deandre Ayton', 0.12187632331412178],
 ['Dwayne Bacon', 0

In [44]:
model = model_builder(train_with_sc, ['true_shooting_percentage', 'win_shares'])
bam_adebayo = predict_2020(data_2020, model, "Bam Adebayo", ['true_shooting_percentage', 'win_shares'])

[[-0.00168966  0.04395077]]
<bound method BaseEstimator.get_params of LinearRegression()>


In [48]:
predict_2020(data_2020, model, "Evan Turner", ['true_shooting_percentage', 'win_shares'])

<bound method BaseEstimator.get_params of LinearRegression()>


0.2604885169264007

In [70]:
predict_all(data_2020, model, ['true_shooting_percentage', 'win_shares'])

[['Steven Adams', 0.3615110705606641],
 ['Bam Adebayo', 0.44942273952973455],
 ['LaMarcus Aldridge', 0.273665298284835],
 ['Kyle Alexander', 0.07600681927311603],
 ['Nickeil Alexander-Walker', 0.06726228695911826],
 ['Grayson Allen', 0.1285635650735589],
 ['Jarrett Allen', 0.4361259923964249],
 ['Kadeem Allen', 0.08038161991831652],
 ['Al-Farouq Aminu', 0.0805793099980406],
 ['Justin Anderson', 0.0718618122248598],
 ['Kyle Anderson', 0.1858262846442101],
 ['Ryan Anderson', 0.07624844048166769],
 ['Giannis Antetokounmpo', 0.563669384968862],
 ['Kostas Antetokounmpo', 0.07972265298590289],
 ['Thanasis Antetokounmpo', 0.08480373100994669],
 ['Carmelo Anthony', 0.13310057296366118],
 ['OG Anunoby', 0.3263572167893627],
 ['Ryan Arcidiacono', 0.13745171838564682],
 ['Trevor Ariza', 0.18572659477494752],
 ['Trevor Ariza', 0.12428000600361569],
 ['Trevor Ariza', 0.13729626977594925],
 ['D.J. Augustin', 0.20337279767144173],
 ['Deandre Ayton', 0.21213929554985328],
 ['Dwayne Bacon', 0.040983069

In [106]:
data_2020[data_2020['name'] == 'Jabari Parker'][from_team]

,value_over_replacement_player,win_shares,player_efficiency_rating,minutes_played,box_plus_minus
480,0.5,1.6,18.0,24.1,0.3
481,0.4,1.2,17.2,26.2,0.0
482,0.1,0.4,26.5,13.3,2.6


In [108]:
train_with_sc.to_excel('Complete_Train_Set_With_Cap_Usage.xlsx')

In [109]:
data_2020.to_excel('Clean_2019_2020_Data.xlsx')